In [35]:
events = ['27_03_2021_16_26_11_task_standard_seed_7', '27_03_2021_16_38_59_task_standard_seed_7', '27_03_2021_16_47_36_task_standard_seed_122', '27_03_2021_16_57_33_task_standard_seed_122', '27_03_2021_17_06_26_task_standard_seed_809', '27_03_2021_17_19_07_task_standard_seed_809']

In [40]:
from tensorboard.backend.event_processing import event_accumulator
import os
import numpy as np
import pandas as pd
import json
pd.options.display.float_format = '{:,.3f}'.format

def load_metric_from_event(event_name): 
    event_path = os.path.join("runs", event_name)
    event_name = os.listdir(event_path)[0]
    event_path = os.path.join(event_path, event_name)

    # only read scalars
    ea = event_accumulator.EventAccumulator(event_path, size_guidance={event_accumulator.SCALARS: 0})

    # load scalars
    ea.Reload()

    return get_best_epoch_metrics(ea)

def get_best_epoch_metrics(ea):
    test_accs = ea.Scalars(f"test/acc")
    test_acc_vals = [acc.value for acc in test_accs]
    test_best_acc_idx = np.argmax(test_acc_vals)
    test_best_acc = np.max(test_acc_vals)
    test_best_epoch = test_accs[test_best_acc_idx].step

    train_accs = ea.Scalars(f"train/acc")
    train_best_acc = train_accs[test_best_acc_idx].value


    try:
        train_img_clas_accs = ea.Scalars(f"train/img_class_acc")
        train_best_img_clas_acc = train_img_clas_accs[test_best_acc_idx].value

        
        test_img_clas_accs = ea.Scalars(f"train/img_class_acc")
        test_best_img_clas_acc = test_img_clas_accs[test_best_acc_idx].value
    except KeyError:
        train_best_img_clas_acc = np.nan
        test_best_img_clas_acc = np.nan
    
    return [test_best_epoch, train_best_acc, test_best_acc, train_best_img_clas_acc, test_best_img_clas_acc]

In [47]:
# choose which params to include in the
include_params = ["task", "game_type", "ic_loss_weight", "num_imgs", "same_class_prob", "seed" ]

In [48]:
results = []
args_dir = os.path.join("args")
for event_name in events:
    # get parameters
    with open(f"{args_dir}\\args_{event_name}.json") as json_file:
        params = json.load(json_file)
    # wrap each value into a list
    params = {key:[value] for key, value in params.items()}

    # create dataframe and add parameters
    results_df = pd.DataFrame(params)[include_params]

    # add metrics
    metric_cols = ["best epoch", "best train acc", "best test acc", "best train img acc", "best test img acc"]
    results_df.loc[:, metric_cols] = load_metric_from_event(event_name)
    # append to final table
    results.append(results_df)


results = pd.concat(results)
# manually add noise ac due to issues with tensorboard
results["noise acc"] = [0.8244, 0.8056, 0.7338, 0.6512, 0.869, 0.7491]
results

,task,game_type,ic_loss_weight,num_imgs,same_class_prob,seed,best epoch,best train acc,best test acc,best train img acc,best test img acc,noise acc
0,standard,SenderReceiverRnnGS,1.000,2,0.500,7,10,0.933,0.954,nan,nan,0.824
0,standard,SenderReceiverRnnReinforce,1.000,2,0.500,7,8,0.872,0.919,nan,nan,0.806
0,standard,SenderReceiverRnnGS,1.000,2,0.500,122,7,0.955,0.970,nan,nan,0.734
0,standard,SenderReceiverRnnReinforce,1.000,2,0.500,122,10,0.828,0.873,nan,nan,0.651
0,standard,SenderReceiverRnnGS,1.000,2,0.500,809,8,0.918,0.942,nan,nan,0.869
0,standard,SenderReceiverRnnReinforce,1.000,2,0.500,809,6,0.879,0.914,nan,nan,0.749


In [49]:
results["success"] = (results["best test acc"]>0.97).astype(int)
results = results.drop(["ic_loss_weight", "num_imgs", "best epoch", "best train acc", "best train img acc"], axis=1)
results = results.groupby(by=["game_type"]).mean().drop(["seed"], axis=1).fillna('-')
results

,same_class_prob,best test acc,best test img acc,noise acc,success
game_type,,,,,
SenderReceiverRnnGS,0.500,0.955,-,0.809,0.333
SenderReceiverRnnReinforce,0.500,0.902,-,0.735,0.000
